# Brian Cepparulo - 12/09/2024



# Lab week 2 - how to manipulate a df

In [ ]:
import pandas as pd
import eurostat
import numpy as np
import os
from datetime import datetime


In [ ]:
%store -r df_gdp # recall the df previously stored

In [ ]:
df_gdp


In [ ]:
df_gdp=df_gdp.sort_values(by=['na_item']) #sort data by GDP component

In [ ]:
l=list(df_gdp ['freq'].unique()) #extract list of unic values within column 'freq'
l


In [ ]:
l=list(df_gdp ['unit'].unique()) #extract list of unic values within column 'unit'
l

In [ ]:
l=list(df_gdp ['s_adj'].unique()) #extract list of unic values within column 's_adj'
l

In [ ]:
df_gdp2=df_gdp.drop(['freq','unit','s_adj'], axis=1) #we define a new df and drop columns

In [ ]:
df_gdp2=df_gdp2.rename(columns={"na_item":"series","geo\TIME_PERIOD":"geo"}) #rename columns

In [ ]:
df_gdp2


In [ ]:
year_quart=list(df_gdp2.columns)
year_quart


In [ ]:
series_l=list(df_gdp2 ['series'].unique())  #extract list of unique items and insert in list
series_l

# change columns name of qurterly data for wide-to-long reshape

In [ ]:
df_gdp2=df_gdp2.rename(columns={c: 'A_'+c for c in df_gdp2.columns if c not in ['geo', 'series']})
for col in df_gdp2.columns:
    df_gdp2.rename(columns={col:col.replace("-Q","")},inplace=True) 
    


In [ ]:
### Reshape df
df_gdp2=pd.wide_to_long(df_gdp2, ['A'], i=['geo','series'], j="time", sep='_', ).reset_index()
df_gdp2=df_gdp2.pivot_table(index=['geo','time'], columns='series', values='A')  #reshape eonomic series wide 
#and drop rows that contain missing values
df_gdp2 = df_gdp2.reset_index()

In [ ]:
is_integer = df_gdp2['time'].apply(lambda x: isinstance(x, int)) #check if 'time' contains integer
is_string = df_gdp2['time'].apply(lambda x: isinstance(x, str)) #check if 'time' contains strings

In [ ]:
is_integer


In [ ]:
all_integers = is_integer.all() # as the list above is truncated we can check if alla values are integers like this
all_integers

In [ ]:
true_count = is_integer.sum()  # Counts how many True values
print(f"True count: {true_count}")


In [ ]:
df_gdp2


# Function to convert "YYYYQ" to quarterly dates

In [ ]:
def convert_to_quarterly_date(quarter_str):
    year = int(str(quarter_str)[:4])
    quarter = int(str(quarter_str)[-1])
    
    # Determine the month based on the quarter
    if quarter == 1:
        month = 1
    elif quarter == 2:
        month = 4
    elif quarter == 3:
        month = 7
    elif quarter == 4:
        month = 10
    
    return datetime(year, month, 1)

# Apply the conversion function to the 'Quarter' column and create a new 'Time' column

In [ ]:
df_gdp2['date'] = df_gdp2['time'].apply(convert_to_quarterly_date)

In [ ]:
df_gdp2.drop(columns=['time'], inplace=True)

In [ ]:
%store df_gdp2